## Context manager (RAII implementation)

In [1]:
#we can't implement raii in python, we dont know what is a constructor, destructor, ecc

help(open)   #OPEN è un context manager

Help on built-in function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise OSError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position

In [2]:
#boh
f = open("new_file.txt","w")
for i in range(10):
    print(i, file=f)
f.close()

In [3]:
#WITH opens context manager
with open("new_file.txt", "w") as f:    # 'w' per Write
    for i in range(10):
        print(i, file=f)          
        #f.write(str(i)+'\n')    #equivalent   
    #(vars inside here are maintained also out)

In [4]:
f.closed    #when we exit the context manager, file is automatically closed

True

In [5]:
i

9

In [6]:
%%timeit

a = []
with open("new_file.txt", "r") as f:
    for line in f:
        a.append(int(line))
a                                        #not pythonic

14.5 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [1]:
%%timeit 

with open("new_file.txt") as f:
    a = [int(line) for line in f]  #its ok to def new vars inside the context manager, tanto restano
a

# (NB: list compr sono piu veloci che un for come nella cella sopra)

FileNotFoundError: [Errno 2] No such file or directory: 'new_file.txt'

## implementiamo il nostro context manager

In [10]:
# adapted from Fluent Python
class LookingGlass:   #Mirror era meglio
    def __enter__(self):  # no other arguments
        print("__enter__")
        import sys
        
        self.lefttoright = sys.stdout.write   #store orig way of writing (instance var aggiunta ora)
        sys.stdout.write = self.reverse_write    # sostituiamola con la nostra func

    def reverse_write(self, text):
        self.lefttoright(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        import sys    #perchè importa di nuovo ??

        sys.stdout.write = self.lefttoright   #restore original handling of exception, if any
        print("__exit__")
        return True    # everything has been managed


with LookingGlass():
    print("What is going on?")
    print("I cannot read this")

print("back to work")

__enter__
?no gniog si tahW
siht daer tonnac I
__exit__
back to work


In [12]:
#EXAM QUESTION: come si implementa RAII?

# con context manager, che è implementato mettendo un enter e un exit